<a href="https://colab.research.google.com/github/Fnxzera/Urban-Mobility-Lisbon/blob/main/pysparkProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder \
    .appName("UrbanMobility") \
    .getOrCreate()


In [ ]:
df = spark.read.csv('Urban_Mobility/infoLisbon.csv',header=True,inferSchema=True)

In [ ]:
df = df.withColumn(('Data_Inicio'),substring('periodos_condicionamentos',45, 10))
df = df.withColumn(('Data_Termino'),substring('periodos_condicionamentos',-68, 10))
df = df.withColumn("Data_Inicio", to_date("Data_Inicio"))
df = df.withColumn("Duração" , df['Data_Termino']-df['Data_Inicio'])

In [ ]:
df = df.drop('position','entity_id','periodos_condicionamentos')

In [ ]:
values_to_fill = {'Duração': 0, 'Data_Termino': '2000-01-01','Data_Inicio':'2000-01-01','creation_date':'2000-01-01',
                 'motivo':'desconhecido','morada':'desconhecida','restricao_circulacao':'desconhecido','impacto':'desconhecido',
                 }
df = df.na.fill(values_to_fill)

In [ ]:
valores_possiveis = ['Relevante', 'Pouco Relevante']
df_filtrado = df.filter((col('impacto')== 'Relevante') | (col('impacto')== 'Pouco relevante'))

In [ ]:
contagem_morada = df_filtrado.groupBy('morada').agg(count('motivo').alias('Quantidade'))  # Conta as ocorrências em cada morada
contagem_Duração = df_filtrado.groupBy('Duração').agg(count('Duração').alias('Quantidade')) # Conta as ocorrências para cada duração
contagem_impacto = df_filtrado.groupBy('impacto').agg(count('impacto').alias('Quantidade')) # Conta as ocorrências para cada impacto (Relev e Pouco Rev)
DMorada = df_filtrado.groupBy("morada").agg(avg("Duração").alias("Duração Média")) # Mostra a duração média para cada morada
DRestricao = df_filtrado.groupBy("restricao_circulacao").agg(avg("Duração").alias("Duração Média")) # Mostra a duração média para cada restrição
DMotivo = df_filtrado.groupBy("motivo").agg(avg("Duração").alias("Duração Média")) # Mostra a duração média para cada motivo

In [ ]:
contagem_motivo.orderBy(desc("Quantidade")).show(25)

+--------------------+----------+
|              motivo|Quantidade|
+--------------------+----------+
|CARGAS E DESCARGA...|      1812|
|BETONAGENS/CARGAS...|      1119|
|OBRA - FAIXA DE R...|       890|
|            PINTURAS|       796|
|           FILMAGENS|       580|
|            MUDANÇAS|       564|
|OBRA - PASSEIO E ...|       506|
|  CARGAS E DESCARGAS|       491|
|        CONCENTRAÇÃO|       358|
|            AUTOGRUA|       327|
|PLANTAÇÃO / PODA ...|       310|
|        desconhecido|       283|
|RESERVA DE ESTACI...|       231|
|          BETONAGENS|       224|
|ACESSO DE VEÍCULO...|       198|
|LIGAÇÃO DE RAMAL ...|       183|
|        MANIFESTAÇÃO|       134|
|LIGAÇÃO DE RAMAL ...|       126|
|     REPAVIMENTAÇÕES|        80|
|              EVENTO|        78|
|SUBSTITUIÇÃO DE C...|        71|
|OBRAS NO SUBSOLO ...|        70|
|    MONTAGEM DE GRUA|        63|
|REP. DE COLECTOR/...|        48|
| DESMONTAGEM DE GRUA|        47|
+--------------------+----------+
only showing t

In [ ]:
contagem_morada.show(25)

+--------------------+----------+
|              morada|Quantidade|
+--------------------+----------+
|Praça Marechal Hu...|         7|
|  Praça Dom Pedro IV|         9|
|Rua dos Lusiadas,...|        18|
|Rua Almirante Bar...|         2|
|Rua do Guarda Mor...|         2|
|Calçada das Neces...|         4|
|Rua Carlos José b...|         1|
|Estrada da Correi...|         1|
|Rua Senhora da Gl...|         1|
| Rua São Paulo,14-16|         1|
|Rua Doutor Teófil...|         1|
|Rua das Portas de...|         1|
|Praça Luis de Camões|         1|
|Rua Cortes Reais,...|         1|
|Avenida Fernando ...|         4|
|   Bairro Padre Cruz|         1|
|Rua Filipe Folque,65|         1|
|Praça das Amoreir...|         2|
|Avenida 24 de Jul...|         1|
|        Eixo Central|         1|
|Av. Gen. Norton M...|         1|
|Rua Eduardo Malta...|         1|
|Rua Almeida Brand...|         2|
|Estação Fluvial- ...|         1|
|Rua das Escolas G...|         1|
+--------------------+----------+
only showing t

In [ ]:
contagem_Duração.orderBy(desc("Quantidade")).show(25)

+-----------------+----------+
|          Duração|Quantidade|
+-----------------+----------+
| INTERVAL '0' DAY|      2355|
|INTERVAL '14' DAY|      1622|
|INTERVAL '13' DAY|      1166|
| INTERVAL '1' DAY|       801|
|INTERVAL '11' DAY|       645|
| INTERVAL '4' DAY|       385|
| INTERVAL '2' DAY|       353|
|INTERVAL '15' DAY|       241|
| INTERVAL '3' DAY|       182|
|INTERVAL '18' DAY|       174|
| INTERVAL '9' DAY|       169|
| INTERVAL '6' DAY|       155|
|INTERVAL '12' DAY|       133|
|INTERVAL '10' DAY|       120|
| INTERVAL '5' DAY|       100|
|INTERVAL '16' DAY|        87|
| INTERVAL '7' DAY|        83|
| INTERVAL '8' DAY|        77|
|INTERVAL '31' DAY|        67|
|INTERVAL '25' DAY|        62|
|INTERVAL '17' DAY|        44|
|INTERVAL '29' DAY|        39|
|INTERVAL '39' DAY|        38|
|INTERVAL '30' DAY|        36|
|INTERVAL '28' DAY|        35|
+-----------------+----------+
only showing top 25 rows



In [ ]:
contagem_impacto.orderBy(desc("Quantidade")).show(25)

+---------------+----------+
|        impacto|Quantidade|
+---------------+----------+
|      Relevante|      6907|
|Pouco relevante|      2842|
+---------------+----------+



In [ ]:
DMotivo.show(30, truncate = False)

+---------------------------------------+-------------------------------------------+
|motivo                                 |Duração Média                              |
+---------------------------------------+-------------------------------------------+
|CARGAS E DESCARGAS/OBRAS               |INTERVAL '14 22:46:05.562914' DAY TO SECOND|
|DESMONTAGEM DE GRUA                    |INTERVAL '0 14:17:52.340426' DAY TO SECOND |
|PINTURAS                               |INTERVAL '12 16:38:35.577889' DAY TO SECOND|
|REPAVIMENTAÇÕES                        |INTERVAL '11 18:00:00' DAY TO SECOND       |
|desconhecido                           |INTERVAL '0 00:00:00' DAY TO SECOND        |
|REPARAÇÃO DE GUARDAS/JUNTAS            |INTERVAL '2 18:00:00' DAY TO SECOND        |
|CARGAS E DESCARGAS                     |INTERVAL '22 02:41:18.207739' DAY TO SECOND|
|LIGAÇÃO DE RAMAL - FAIXA DE RODAGEM    |INTERVAL '12 14:25:34.42623' DAY TO SECOND |
|MUDANÇAS                               |INTERVAL '1 1

In [ ]:
DRestricao.show(30, truncate = False)

+--------------------+-------------------------------------------+
|restricao_circulacao|Duração Média                              |
+--------------------+-------------------------------------------+
|Corte total         |INTERVAL '9 15:43:12.686804' DAY TO SECOND |
|Mantém perfil de via|INTERVAL '15 05:31:56.652994' DAY TO SECOND|
|Estacionamento      |INTERVAL '11 20:37:43.366337' DAY TO SECOND|
|Corte num sentido   |INTERVAL '10 00:00:00' DAY TO SECOND       |
|Cortes temporários  |INTERVAL '9 07:50:31.238779' DAY TO SECOND |
|Estreitamento de via|INTERVAL '14 00:14:16.023418' DAY TO SECOND|
|Passeio             |INTERVAL '8 13:50:51.539708' DAY TO SECOND |
+--------------------+-------------------------------------------+



In [ ]:
DMorada.show(50, truncate = False)

+---------------------------------------------------------------------------------------------------+-------------------------------------------+
|morada                                                                                             |Duração Média                              |
+---------------------------------------------------------------------------------------------------+-------------------------------------------+
|Praça Marechal Humberto Delgado                                                                    |INTERVAL '17 17:08:34.285714' DAY TO SECOND|
|Praça Dom Pedro IV                                                                                 |INTERVAL '2 16:00:00' DAY TO SECOND        |
|Rua dos Lusiadas, nº 29                                                                            |INTERVAL '11 10:40:00' DAY TO SECOND       |
|Rua Almirante Barroso, 18                                                                          |INTERVAL '13 12:00:00' 

In [ ]:
df_filtrado.describe().show()

+-------+---------------+--------------------+--------------------+--------------------+-------------------+------------+
|summary|        impacto|restricao_circulacao|              morada|              motivo|      creation_date|Data_Termino|
+-------+---------------+--------------------+--------------------+--------------------+-------------------+------------+
|  count|           9749|                9749|                9749|                9749|               9749|        9749|
|   mean|           NULL|                NULL|        5.06164659E8|                NULL|               NULL|        NULL|
| stddev|           NULL|                NULL|                NULL|                NULL|               NULL|        NULL|
|    min|Pouco relevante|   Corte num sentido|    Av. Cidade de...|ACESSO DE VEÍCULO...|         2000-01-01|  2000-01-01|
|    max|      Relevante|             Passeio|Área delimitada p...|        desconhecido|2021-08-31 18:36:46|  2026-07-20|
+-------+---------------

In [ ]:
# MEDIA DE